In [ ]:
import basic
import os
import flopy.utils.mflistfile as mflist

In [ ]:

m = basic.load_model(verbose = True)

In [ ]:
# success, other = m.run_model( silent=False)

In [ ]:
swr = os.path.join(m.model_ws, 'RR.SWR')

In [ ]:
def write_swr(DLENCONV = 3.28081 ,
TIMECONV = 1 ,
RTINI = 1 ,
RTMIN = 0.1 ,
RTMAX = 360 ,
RTPRN = 1 ,
RTMULT = 1.01 ,
NTMULT = 10 ,
DMINGRAD = 0.000000000001 ,
DMNDEPTH = 0.001 ,
DMAXRAI = 0.1524 ,
DMAXSTG = 0.2 ,
DMAXINF = 0.01):
    
    with open(swr, 'r', encoding='utf-8') as file:
        data = file.readlines()
    inputs = f"{ DLENCONV }\t{ TIMECONV }\t{ RTINI }\t{ RTMIN }\t{ RTMAX }\t{ RTPRN }\t{ RTMULT }\t{ NTMULT }\t{ DMINGRAD }\t{ DMNDEPTH }\t{ DMAXRAI }\t{ DMAXSTG }\t{ DMAXINF }\n"

    # inp = inputs.format(DLENCONV, TIMECONV, RTINI, RTMIN, RTMAX, RTPRN, RTMULT, NTMULT, DMINGRAD, DMNDEPTH, DMAXRAI, DMAXSTG, DMAXINF)
    
    data[20] = inputs
    # data[20] = '     3.28081     1.0      30     .1     360.000  1.0E+00   1.01     10  1.0E-12  1.0E-03 0.15240    0.20     .01 \n'

    with open(swr, 'w', encoding='utf-8') as file:
        file.writelines(data)
        
write_swr()


In [ ]:
def get_final(variable, value):
    plist = os.path.join(m.model_ws,  'RRlist.lst')


    mf_list = mflist.MfListBudget(plist)
    
    last_date = mf_list.idx_map[-1]
    
    inst, cum = mf_list.get_dataframes()
    mf_disc = cum.at[cum.index[-1], 'PERCENT_DISCREPANCY']
    
    runtime = mf_list.get_model_runtime()
    
    
    swr_list = mflist.SwrListBudget(plist)   
    inst, cum = swr_list.get_dataframes()
    swr_disc = cum.at[cum.index[-1], 'PERCENT_DISCREPANCY']
    
    output = {'last_date':last_date, 'mf_disc':mf_disc, 'swr_disc':swr_disc,
              'runtime':runtime,'variable':variable, 'value':value}
    
    return output
    
get_final('test', 0)

In [ ]:

vals = {
        'RTMAX': [3000],
       'RTMULT':[1.05,1.1, 1.15, 1.3],
       'NTMULT': [5, 20,50,100],
       'DMAXINF':[0.1, 1, 10]}


stack ={}
stack[0] = {'last_date': [24, 1, 2995412],
 'mf_disc': 3.3,
 'swr_disc': -0.05,
 'runtime': 580.503,
 'variable': 'test',
 'value': 0}

run = 1

for key in vals.keys():
    
    # success, buffer = m.run_model(silent =True)
    # out = get_final('base_case', -999)
    
    for number in vals[key]:
        print(f"running: {key}=={number}")
        eval(f"write_swr({key}={number})")
        success, buffer = m.run_model(silent =True)
        if success:
            out = get_final(key, number)

            stack[run] = out
            print(out)
        else:
            stack[run] = {'last_date': [0, 0,999],
                         'mf_disc': 999,
                         'runtime': 0,
                         'variable': key,
                         'value': 'failed'}
            
        
        print(stack)
        run = run+1

In [ ]:
      2    100  100     10 1.0E-09    .001     .10      .5    .50      0      0   2  10 1 1 ## this does  fail

(0, {'last_date': [24, 1, 2995412], 'mf_disc': 3.3, 'swr_disc': -0.05, 'runtime': 580.503, 'variable': 'test', 'value': 0})
(1, {'last_date': [0, 0, 999], 'mf_disc': 999, 'runtime': 0, 'variable': 'RTMAX', 'value': 'failed'})
(2, {'last_date': [0, 0, 999], 'mf_disc': 999, 'runtime': 0, 'variable': 'RTMULT', 'value': 'failed'})
(3, {'last_date': [0, 0, 999], 'mf_disc': 999, 'runtime': 0, 'variable': 'RTMULT', 'value': 'failed'})
(4, {'last_date': [0, 0, 999], 'mf_disc': 999, 'runtime': 0, 'variable': 'RTMULT', 'value': 'failed'})
(5, {'last_date': [0, 0, 999], 'mf_disc': 999, 'runtime': 0, 'variable': 'RTMULT', 'value': 'failed'})
(6, {'last_date': [0, 0, 999], 'mf_disc': 999, 'runtime': 0, 'variable': 'NTMULT', 'value': 'failed'})
(7, {'last_date': [24, 1, 3145601], 'mf_disc': 3.34, 'swr_disc': -0.05, 'runtime': 794.533, 'variable': 'NTMULT', 'value': 20})
(8, {'last_date': [24, 1, 3373989], 'mf_disc': 3.29, 'swr_disc': -0.05, 'runtime': 1144.378, 'variable': 'NTMULT', 'value': 50})
(9, {'last_date': [24, 1, 3591562], 'mf_disc': 3.28, 'swr_disc': -0.05, 'runtime': 1630.899, 'variable': 'NTMULT', 'value': 100})
(10, {'last_date': [0, 0, 999], 'mf_disc': 999, 'runtime': 0, 'variable': 'DMAXINF', 'value': 'failed'})
(11, {'last_date': [0, 0, 999], 'mf_disc': 999, 'runtime': 0, 'variable': 'DMAXINF', 'value': 'failed'})
(12, {'last_date': [0, 0, 999], 'mf_disc': 999, 'runtime': 0, 'variable': 'DMAXINF', 'value': 'failed'})


In [ ]:
[print(item) for item in stack.items()]

In [ ]:
ISOLVER NOUTER NINNER IBT    TOLS     TOLR    TOLA DAMPSS DAMPTR IPRSWR MUTSWR IPC NLEVELS DROPTOL IBTPRT

In [ ]:
def write_solv(ISOLVER = 2 ,
NOUTER = 100 ,
NINNER = 100 ,
IBT = 10 ,
TOLS = 0.000000001 ,
TOLR = 500 ,
TOLA = 0.1 ,
DAMPSS = 1 ,
DAMPTR = 1 ,
IPRSWR = 0 ,
MUTSWR = 0 ,
IPC = 2 ,
):
    
    with open(swr, 'r', encoding='utf-8') as file:
        data = file.readlines()
    inputs = f"{ ISOLVER }\t{ NOUTER }\t{ NINNER }\t{ IBT }\t{ TOLS }\t{ TOLR }\t{ TOLA }\t{ DAMPSS }\t{ DAMPTR }\t{ IPRSWR }\t{ MUTSWR }\t{ IPC } \n"
    
    data[26] = inputs


    with open(swr, 'w', encoding='utf-8') as file:
        file.writelines(data)
        
write_solv()

In [ ]:

vals = {
        'TOLS':[2e-9, 8e-9,1e-8,2e-8],
        'TOLR':[ 500*1.2,500*2, 500*5] ,
        'TOLA':[0.01, 0.1*1.2, .1*2, .1*5] ,
        }


stack ={}


run = 1

for key in vals.keys():
    print('running basecase')
    success, buffer = m.run_model(silent =True)
    out = get_final('base_case', -999)
    
    for number in vals[key]:
        print(f"running: {key}=={number}")
        eval(f"write_solv({key}={number})")
        success, buffer = m.run_model(silent =True)
        if success:
            out = get_final(key, number)

            stack[run] = out
            print(out)
            print('-------'*5)
        else:
            stack[run] = {'last_date': [-999, -999,-999],
                         'mf_disc': -999,
                         'runtime': number,
                         'variable': key,
                         'value': 'failed'}
            
        
        # print(stack)
        run = run+1

In [ ]:
def write_solv(ISOLVER = 2 ,
NOUTER = 100 ,
NINNER = 100 ,
IBT = 10 ,
TOLS = 0.000000001 ,
TOLR = 500 ,
TOLA = 0.1 ,
DAMPSS = 1 ,
DAMPTR = 1 ,
IPRSWR = 0 ,
MUTSWR = 0 ,
IPC = 2 ,
):
    
    with open(swr, 'r', encoding='utf-8') as file:
        data = file.readlines()
    inputs = f"{ ISOLVER }\t{ NOUTER }\t{ NINNER }\t{ IBT }\t{ TOLS }\t{ TOLR }\t{ TOLA }\t{ DAMPSS }\t{ DAMPTR }\t{ IPRSWR }\t{ MUTSWR }\t{ IPC } \n"
    
    data[26] = inputs


    with open(swr, 'w', encoding='utf-8') as file:
        file.writelines(data)
        
write_solv()

In [ ]:

vals = {

        'TOLR':[ 0.001*.1, 0.001*1.2,.001*2, .001*5] ,
        'TOLA':[0.01*.1, 0.1*10, .1*100, .1*5] ,
        'DAMPTR' :[.5,.75, 1, 1.25, 1.5], 
    'IPC':[1,2,3]
        }


stack ={}


run = 1
success, buffer = m.run_model(silent =True)
out = get_final('base_case', -999)
stack[run] = out

for key in vals.keys():
    print('running basecase')

    
    for number in vals[key]:
        print(f"running: {key}=={number}")
        eval(f"write_solv({key}={number})")
        success, buffer = m.run_model(silent =True)
        if success:
            out = get_final(key, number)

            stack[run] = out
            print(out)
            print('-------'*5)
        else:
            stack[run] = {'last_date': [-999, -999,-999],
                         'mf_disc': -999,
                         'runtime': number,
                         'variable': key,
                         'value': 'failed'}
            
        
        # print(stack)
        run = run+1

In [ ]:

vals = {

        'TOLR':[ 0.001*10, 0.001*20,.001*50, .001*100, .001*150, .001*500, .001*1000] ,

        }


stack ={}


run = 1
success, buffer = m.run_model(silent =True)
out = get_final('base_case', -999)
stack[run] = out

for key in vals.keys():
    print(f'running {key}')

    
    for number in vals[key]:
        print(f"running: {key}=={number}")
        eval(f"write_solv({key}={number})")
        success, buffer = m.run_model(silent =True)
        if success:
            out = get_final(key, number)

            stack[run] = out
            print(out)
            print('-------'*5)
        else:
            stack[run] = {'last_date': [-999, -999,-999],
                         'mf_disc': -999,
                         'runtime': number,
                         'variable': key,
                         'value': 'failed'}
            
        
        # print(stack)
        run = run+1

In [ ]:
write_solv(TOLR = 0.5)